# 建立并训练网络

In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from six.moves import cPickle as pickle
from six.moves import range
from RNNCell_diy import diyLSTMCell
from six.moves import cPickle as pickle

%matplotlib inline

读取训练数据

In [2]:
pfile = './concat_image_train_data.pickle'
with open(pfile, "rb") as f:
    train_data = pickle.load(f)
train_dataset = train_data["train_dataset"]
train_dataset.shape = train_dataset.shape + (1,)
train_labels = train_data["train_labels"]
train_labels_num = train_data["train_labels_num"]

In [3]:
cdic = {"a":0, "b":1, "c":2, "d":3, "e":4, "f":5, "g":6, "h":7, "i":8, "j":9, " ":10}
cdic_r = {}
for key, value in cdic.items():
    cdic_r[value] = key

In [4]:
image_size_h = 56
image_size_w = 112
num_channels = 1

构建网络结构

In [11]:
batch_size = 50
kernel_size = 5
pooling_size = 2
channels_one = 12
channels_two = 32
hidden1_size = 300
hidden2_size = 260
dropout = 0.5
lam = 0.01
max_time_step = 3
rnn_num_nodes = 260
rnn_num_layers = 3


graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size_h, image_size_w, num_channels))
    tf_train_labels = tf.placeholder(tf.int32,shape=(batch_size, max_time_step))
    tf_train_labels_num = tf.placeholder(tf.int32,shape=(batch_size))
    
    #variables
    layer_weight1 = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, num_channels, channels_one],stddev=0.1))
    layer_biases1 = tf.Variable(tf.zeros([channels_one]), name="bias1")
    
    layer_weight3 = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, channels_one, channels_two], stddev=0.1))
    layer_biases3 = tf.Variable(tf.constant(1.0, shape=[channels_two]), name="bias3")
    
    neuron_num = ((((image_size_h-pooling_size)//pooling_size+1-kernel_size+1-pooling_size)//pooling_size+1)**2)*channels_two
    layer_weight5 = tf.Variable(tf.truncated_normal([neuron_num, hidden1_size], stddev=0.1))
    layer_biases5 = tf.Variable(tf.constant(1.0, shape=[hidden1_size]), name="bias5")
    
    layer_weight6 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))
    layer_biases6 = tf.Variable(tf.constant(1.0, shape=[hidden2_size]), name="bias6")
    
    
    # Classifier weights and biases.
    s_weight = tf.Variable(tf.truncated_normal([hidden2_size, max_time_step], stddev=0.1))
    s_biases = tf.Variable(tf.constant(1.0, shape=[max_time_step]), name="s_bias")
    
    c1_weight = tf.Variable(tf.truncated_normal([hidden2_size, len(cdic)], stddev=0.1))
    c1_biases = tf.Variable(tf.constant(1.0, shape=[len(cdic)]), name="c1_bias")
    
    c2_weight = tf.Variable(tf.truncated_normal([hidden2_size, len(cdic)-1], stddev=0.1))
    c2_biases = tf.Variable(tf.constant(1.0, shape=[len(cdic)-1]), name="c2_bias")
    
    c3_weight = tf.Variable(tf.truncated_normal([hidden2_size, len(cdic)-1], stddev=0.1))
    c3_biases = tf.Variable(tf.constant(1.0, shape=[len(cdic)-1]), name="c3_bias")
    
    
    
#     w = tf.Variable(tf.truncated_normal([rnn_num_nodes, len(cdic)], -0.1, 0.1))
#     b = tf.Variable(tf.zeros([len(cdic)]), name="bias_o")
    
    # Model
    def conv_model(data):
        conv = tf.nn.conv2d(data, layer_weight1, [1,1,1,1], padding="SAME")+layer_biases1
        pooling = tf.nn.relu(tf.nn.max_pool(conv, [1,2,4,1], [1,2,4,1], padding="VALID"))
        conv = tf.nn.conv2d(pooling, layer_weight3, [1,1,1,1], padding="VALID")+layer_biases3
        pooling = tf.nn.relu(tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding="VALID"))
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden1 = tf.nn.dropout(tf.matmul(reshape, layer_weight5)+layer_biases5, dropout)
        hidden2 = tf.nn.relu(tf.matmul(hidden1, layer_weight6)+layer_biases6)
        return hidden2
            
#     def rnn_model1(input_data):
#         cell_list = []
#         for i in range(rnn_num_layers):
#             rnn_cell = diyLSTMCell(rnn_num_nodes)
#             rnn_cell = tf.contrib.rnn.DropoutWrapper(cell=rnn_cell, input_keep_prob=(1.0 - dropout))
#             cell_list.append(rnn_cell)
#         rnn_cells = tf.contrib.rnn.MultiRNNCell(cell_list)
#         with tf.variable_scope("rnn1"):
#             rnn_outputs, rnn_state = tf.nn.dynamic_rnn(
#                 rnn_cells,
#                 input_data,
#                 dtype=tf.float32)
#         return rnn_outputs, rnn_state
    
#     def rnn_model2(input_data, init_state):
#         cell_list = []
#         for i in range(rnn_num_layers):
#             rnn_cell = diyLSTMCell(rnn_num_nodes)
#             rnn_cell = tf.contrib.rnn.DropoutWrapper(cell=rnn_cell, input_keep_prob=(1.0 - dropout))
#             cell_list.append(rnn_cell)
#         rnn_cells = tf.contrib.rnn.MultiRNNCell(cell_list)
#         with tf.variable_scope("rnn2"):
#             rnn_outputs, rnn_state = tf.nn.dynamic_rnn(
#                 rnn_cells,
#                 input_data,
#                 dtype=tf.float32,
#                 initial_state=init_state,
#                 sequence_length=tf_train_labels_num+1,
#                 swap_memory=True)
#         return rnn_outputs, rnn_state
        
    
    #loss
    conv_out = conv_model(tf_train_dataset)
#     conv_out = tf.reshape(conv_out,[batch_size,1,-1])
#     rnn1_output, rnn1_state = rnn_model1(conv_out)
#     input_data = tf.concat((tf.constant(cdic["remain"],shape=[batch_size,1]), tf_train_labels),1)
#     input_data = tf.one_hot(input_data,len(cdic))
#     tf_train_labels_c = tf.concat((tf_train_labels,tf.constant(cdic["remain"],shape=[batch_size,1])),1)
#     tf_train_labels_c = tf.one_hot(tf_train_labels_c,len(cdic))
#     rnn2_outputs, _ = rnn_model2(input_data, rnn1_state)
#     rnn2_outputs = tf.reshape(rnn2_outputs,[-1, rnn_num_nodes])
#     logits = tf.reshape(tf.matmul(rnn2_outputs, w) + b, [batch_size, max_time_step+1,-1])

    tf_train_labels_c1 = tf_train_labels[:,0]
    tf_train_labels_c1 = tf.one_hot(tf_train_labels_c1, len(cdic))
    c1_logits = tf.matmul(conv_out, c1_weight)+c1_biases
    
    tf_train_labels_c2 = tf_train_labels[:,1]
    mask = tf.logical_not(tf.equal(tf_train_labels_c2, cdic[" "])) # boolean tensor, mask[i] = True iff x[i] ==cdic[" "]
    tf_train_labels_c2 = tf.boolean_mask(tf_train_labels_c2, mask)
    tf_train_labels_c2 = tf.one_hot(tf_train_labels_c2, len(cdic)-1)
#     c2_logits = tf.matmul(conv_out, c2_weight)+c2_biases
    c2_logits = tf.matmul(tf.boolean_mask(conv_out, mask), c2_weight)+c2_biases

    
    tf_train_labels_c3 = tf_train_labels[:,2]
    mask = tf.logical_not(tf.equal(tf_train_labels_c3, cdic[" "])) # boolean tensor, mask[i] = True iff x[i] ==cdic[" "]
    tf_train_labels_c3 = tf.boolean_mask(tf_train_labels_c3, mask)
    tf_train_labels_c3 = tf.one_hot(tf_train_labels_c3, len(cdic)-1)
#     c3_logits = tf.matmul(conv_out, c3_weight)+c3_biases
    c3_logits = tf.matmul(tf.boolean_mask(conv_out, mask), c3_weight)+c3_biases
    
    tf_train_labels_s = tf.one_hot(tf_train_labels_num, max_time_step)
    s_logits = tf.matmul(conv_out, s_weight)+s_biases
    
    tv = tf.trainable_variables()
    regularization_cost = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv if not("bias" in v.name)])
    
    mu3 = tf.cast(tf.shape(tf_train_labels_c3)[0], tf.float32)/batch_size


    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_c1, logits=c1_logits)) \
        +tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_c2, logits=c2_logits)) \
        +tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_c3, logits=c3_logits)) \
        +0.1*tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_s, logits=s_logits)) \
        +lam*regularization_cost
        
#     loss = mu3*(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_c3, logits=c3_logits))+lam*regularization_cost)
#     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_c2, logits=c2_logits)) \
#         +tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels_s, logits=s_logits)) \
#         +lam*regularization_cost
    
    # Optimizer.
#     global_step = tf.Variable(0)
#     learning_rate = tf.train.exponential_decay(
#       10.0, global_step, 100, 0.95, staircase=True)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#     gradients, v = zip(*optimizer.compute_gradients(loss))
#     gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
#     optimizer = optimizer.apply_gradients(
#       zip(gradients, v), global_step=global_step)
    learning_rate = tf.constant(1e-7)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    optimizer = optimizer.minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction1 = tf.nn.softmax(c1_logits)
    train_prediction2 = tf.nn.softmax(c2_logits)
    train_prediction3 = tf.nn.softmax(c3_logits)
    train_prediction_s = tf.nn.softmax(s_logits)
    
    #model save
    tv = tf.trainable_variables()
    print(tv)
    saver = tf.train.Saver()

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 12) dtype=float32_ref>, <tf.Variable 'bias1:0' shape=(12,) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(5, 5, 12, 32) dtype=float32_ref>, <tf.Variable 'bias3:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(4608, 300) dtype=float32_ref>, <tf.Variable 'bias5:0' shape=(300,) dtype=float32_ref>, <tf.Variable 'Variable_3:0' shape=(300, 260) dtype=float32_ref>, <tf.Variable 'bias6:0' shape=(260,) dtype=float32_ref>, <tf.Variable 'Variable_4:0' shape=(260, 3) dtype=float32_ref>, <tf.Variable 's_bias:0' shape=(3,) dtype=float32_ref>, <tf.Variable 'Variable_5:0' shape=(260, 11) dtype=float32_ref>, <tf.Variable 'c1_bias:0' shape=(11,) dtype=float32_ref>, <tf.Variable 'Variable_6:0' shape=(260, 10) dtype=float32_ref>, <tf.Variable 'c2_bias:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'Variable_7:0' shape=(260, 10) dtype=float32_ref>, <tf.Variable 'c3_bias:0' shape=(10,) dtype=float32_ref>]


In [6]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

第一次训练

In [31]:
sess = tf.Session(graph=graph)
with graph.as_default():
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    mean_loss = 0
    step = 0
    l = 100
    for step in range(7001):
        feed_dict = {}
        batch_index = np.random.randint(0,len(train_dataset), batch_size)
        feed_dict[tf_train_dataset] = train_dataset[batch_index]
        feed_dict[tf_train_labels] = train_labels[batch_index]
        feed_dict[tf_train_labels_num] = train_labels_num[batch_index]-1
        _,l, predictions1, predictions2, predictions3, predictions_s, lr, labels1, labels2, labels3, labels_s = sess.run(
          [optimizer, loss, train_prediction1, train_prediction2, train_prediction3, train_prediction_s,
           learning_rate,
           tf_train_labels_c1, tf_train_labels_c2, tf_train_labels_c3, tf_train_labels_s], feed_dict=feed_dict)
        mean_loss += l
        if step % 100 == 0:
            if step > 0:
                mean_loss = mean_loss / 100
            # The mean loss is an estimate of the loss over the last few batches.
            print(
            'Average loss at step %d: %f learning rate: %.10f' % (step, mean_loss, lr))
            mean_loss = 0
            print('Minibatch accuracy_s: %.2f%%' % accuracy(predictions_s, labels_s))
            print('Minibatch accuracy1: %.2f%%' % accuracy(predictions1, labels1))
            print('Minibatch accuracy2: %.2f%%' % accuracy(predictions2, labels2))
            print('Minibatch accuracy3: %.2f%%' % accuracy(predictions3, labels3))

Average loss at step 0: 125.949852 learning rate: 0.0099999998
Minibatch accuracy_s: 54.00%
Minibatch accuracy1: 10.00%
Minibatch accuracy2: 2.00%
Minibatch accuracy3: 8.00%
Average loss at step 100: 65.065210 learning rate: 0.0099999998
Minibatch accuracy_s: 46.00%
Minibatch accuracy1: 12.00%
Minibatch accuracy2: 14.00%
Minibatch accuracy3: 9.52%
Average loss at step 200: 20.643609 learning rate: 0.0099999998
Minibatch accuracy_s: 52.00%
Minibatch accuracy1: 4.00%
Minibatch accuracy2: 10.00%
Minibatch accuracy3: 3.85%
Average loss at step 300: 12.250149 learning rate: 0.0099999998
Minibatch accuracy_s: 50.00%
Minibatch accuracy1: 14.00%
Minibatch accuracy2: 18.00%
Minibatch accuracy3: 9.52%
Average loss at step 400: 10.270074 learning rate: 0.0099999998
Minibatch accuracy_s: 58.00%
Minibatch accuracy1: 16.00%
Minibatch accuracy2: 8.00%
Minibatch accuracy3: 7.41%
Average loss at step 500: 9.508391 learning rate: 0.0099999998
Minibatch accuracy_s: 54.00%
Minibatch accuracy1: 6.00%
Minib

In [32]:
saver.save(sess,"model3/model.ckpt")
sess.close()

使用已训练的模型继续训练

In [12]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess,"model3/model.ckpt")
    mean_loss = 0
#     reset_step = global_step.assign(500)
#     reset_step.eval()
    for step in range(2001):
        feed_dict = {}
        batch_index = np.random.randint(0,len(train_dataset), batch_size)
        feed_dict[tf_train_dataset] = train_dataset[batch_index]
        feed_dict[tf_train_labels] = train_labels[batch_index]
        feed_dict[tf_train_labels_num] = train_labels_num[batch_index]-1
        _,l, predictions1, predictions2, predictions3, predictions_s, lr, labels1, labels2, labels3, labels_s = sess.run(
          [optimizer, loss, train_prediction1, train_prediction2, train_prediction3, train_prediction_s,
           learning_rate,
           tf_train_labels_c1, tf_train_labels_c2, tf_train_labels_c3, tf_train_labels_s], feed_dict=feed_dict)
        mean_loss += l
        if step % 100 == 0:
            if step > 0:
                mean_loss = mean_loss / 100
            # The mean loss is an estimate of the loss over the last few batches.
            print(
            'Average loss at step %d: %f learning rate: %.10f' % (step, mean_loss, lr))
            mean_loss = 0
            print('Minibatch accuracy_s: %.2f%%' % accuracy(predictions_s, labels_s))
            print('Minibatch accuracy1: %.2f%%' % accuracy(predictions1, labels1))
            print('Minibatch accuracy2: %.2f%%' % accuracy(predictions2, labels2))
            print('Minibatch accuracy3: %.2f%%' % accuracy(predictions3, labels3))
    saver.save(sess,"model3/model.ckpt")

INFO:tensorflow:Restoring parameters from model3/model.ckpt
Average loss at step 0: 2.522596 learning rate: 0.0000001000
Minibatch accuracy_s: 96.00%
Minibatch accuracy1: 84.00%
Minibatch accuracy2: 82.00%
Minibatch accuracy3: 87.10%
Average loss at step 100: 2.305898 learning rate: 0.0000001000
Minibatch accuracy_s: 98.00%
Minibatch accuracy1: 88.00%
Minibatch accuracy2: 82.00%
Minibatch accuracy3: 80.00%
Average loss at step 200: 2.351716 learning rate: 0.0000001000
Minibatch accuracy_s: 98.00%
Minibatch accuracy1: 90.00%
Minibatch accuracy2: 94.00%
Minibatch accuracy3: 92.00%
Average loss at step 300: 2.311377 learning rate: 0.0000001000
Minibatch accuracy_s: 98.00%
Minibatch accuracy1: 88.00%
Minibatch accuracy2: 84.00%
Minibatch accuracy3: 92.00%
Average loss at step 400: 2.308612 learning rate: 0.0000001000
Minibatch accuracy_s: 100.00%
Minibatch accuracy1: 82.00%
Minibatch accuracy2: 80.00%
Minibatch accuracy3: 91.67%
Average loss at step 500: 2.291946 learning rate: 0.000000100